In [12]:
from icevision.all import *

In [23]:
import io
from PIL import Image
import ipywidgets as widgets
from fastai.vision.all import PILImage
from IPython.display import Markdown, display

In [ ]:
display(Markdown("****Retail Item Detection****"))

In [ ]:
display(Markdown("*****Note:*****"))

In [ ]:
display(Markdown("**1) It can detect objects of classes [ milk_bottle, carton, can, water_bottle,pepsi can].**"))

In [ ]:
display(Markdown("**2) First upload the image using 'Upload button'.**"))

In [ ]:
display(Markdown("**3) Secondly click on the 'Detect' button to predict the object.**"))

In [24]:
#image_dir = "/content/gdrive/MyDrive/Colab Notebooks/Retail-Object-Detection-with-icevision-fastai/Untitled folder/"
btn_image = widgets.FileUpload()
out_pl = widgets.Output()
out_pl.clear_output()
out_img = widgets.Output()
out_img.clear_output()

btn_run = widgets.Button(description="Detect")

In [25]:
class_map = ClassMap(["milk_bottle", "carton", "can", "water_bottle","pepsi can"])

In [26]:
clas=class_map

In [27]:
def det(img_size=384, classes=clas):

    #source, weights, view_img, save_txt, imgsz = source, weights, view_img, save_txt, img_size
    extra_args = {}
    image_size = 384
    model_type = models.ultralytics.yolov5
    backbone = model_type.backbones.small
 
    extra_args['img_size'] = image_size

    model_type, backbone, extra_args
    model = model_type.model(backbone=backbone(pretrained=True), num_classes=len(classes), **extra_args) 


    checkpoint = torch.load('pepsi_can_l.pth', map_location=torch.device('cpu')) # Load the model file, pt, pth file can be;
    if torch.cuda.device_count() > 1:
            # If there are multiple GPUs, parallelize the model and use DataParallel to operate. This process adds a "module.***" to the key value.
      model = nn.DataParallel(model) 
    model.load_state_dict(checkpoint)

    imgs = PILImage.create(btn_image.data[-1])
    imgs.to_thumb(256)

    infer_ds = Dataset.from_images([np.array(imgs)], tfms.A.Adapter([*tfms.A.resize_and_pad(384), tfms.A.Normalize()]),class_map=classes)
    preds = model_type.predict(model, infer_ds, keep_images=True)
    display(show_preds(preds,show=True))

In [28]:
def on_click_classify(change):
    #thumb_size = 460
    img = btn_image.data[-1] # bytes code
    img = Image.open(io.BytesIO(img))
    #img.save(image_dir +'img.jpg')
    #img.thumbnail((thumb_size,thumb_size))
    #img.save(image_dir +"img_thumb.jpg")
    out_pl.clear_output()
    # with out_pl:
    #     img2=Image.open(image_dir +"img_thumb.jpg")
    #     display(img2)
    
    with out_img:    
    
        s_im = det(classes=clas)
        #out_img.clear_output()


def on_upload_display(change):
    img = btn_image.data[-1] # bytes code
    img = Image.open(io.BytesIO(img))
    #img.save('img_orig.jpg')
    out_pl.clear_output()
    with out_pl:
        display(img)
        #display(img.thumbnail((256,256)))

In [33]:
btn_run.on_click(on_click_classify)
#btn_image.on_change(on_upload_display)
# title = widgets.Label("Retail Item Detection")
# note=widgets.Label("Note:")
# note1=widgets.Label("1) It can detect objects of classes [ milk_bottle, carton, can, water_bottle,pepsi can].")
# note2=widgets.Label("2) First upload the image using button.")
# note3=widgets.Label("3) Secondly click on the detect button to predict the object.")
instruction_labl = widgets.Label("Upload your image")
Run_labl = widgets.Label("Click on Detect for Prediction")
#input_widgets = widgets.VBox([title, note, note1,note2, note3, instruction_labl, btn_image,Run_labl, widgets.HBox([btn_run])])
input_widgets = widgets.VBox([instruction_labl, btn_image,Run_labl, widgets.HBox([btn_run])])
output_widgets = widgets.HBox([out_pl, out_img])

In [34]:
widgets.VBox([input_widgets, output_widgets])